In [ ]:
import os
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import col, from_json, to_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Netoyage des variables d'env qui peuvent casser le bind du driver
for var in ["SPARK_LOCAL_IP", "SPARK_DRIVER_BIND_ADDRESS", "SPARK_DRIVER_HOST"]:
    os.environ.pop(var, None)

# Packages Kafka pour Spark (adapter la version si ton pyspark n'est pas 3.5.*)
EXTRA_PACKAGES = [
    "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.2",
]

builder = (
    SparkSession.builder
    .appName("SmartTech-Streaming-Silver")
    .master("local[*]")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "localhost")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
)

# on passe Kafka via extra_packages
spark = configure_spark_with_delta_pip(builder, extra_packages=EXTRA_PACKAGES).getOrCreate()
spark.sparkContext.setLogLevel("WARN") # permet de limiter les logs Spark à l’essentiel, en masquant les messages INFO et DEBUG
# spark.sparkContext.setLogLevel("ALL")
# spark.sparkContext.setLogLevel("DEBUG")
# spark.sparkContext.setLogLevel("INFO")
# spark.sparkContext.setLogLevel("WARN")   # le plus courant
# spark.sparkContext.setLogLevel("ERROR")
# spark.sparkContext.setLogLevel("FATAL")
# spark.sparkContext.setLogLevel("OFF")



  <h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> 1 - Lecture d'un flux</h2>
 
  La méthode `SparkSession.readStream` renvoie un `DataStreamReader` utilisé pour configurer le flux.
 
  Il y a un certain nombre de points clés pour la configuration d'un `DataStreamReader` :
  * Le schéma
  * Le type de flux : Fichiers, Kafka, TCP/IP, etc.
  * Configuration spécifique au type de flux
    * Pour les fichiers, le type de fichier, le chemin vers les fichiers, le nombre maximum de fichiers, etc.
    * Pour TCP/IP, l'adresse du serveur, le numéro de port, etc.
    * Pour Kafka, l'adresse du serveur, le port, les topics, les partitions, etc.


In [2]:
dataSchema = "Arrival_Time long, Creation_Time long, Device string, Index long, Model string, User string, gt string, x double, y double, z double"


In [ ]:
dataPath = "..."
  .readStream                       # Renvoie DataStreamReader
  .option("maxFilesPerTrigger", 1)  # Forcer le traitement d'un seul fichier par déclencheur
  .schema(dataSchema)               # Requis pour tous les DataFrames en streaming
  .json(dataPath)                   # Le répertoire source du flux et le type de fichier
)


Et avec le `DataFrame` initial, nous pouvons appliquer quelques transformations :

In [6]:
streamingDF = (initialDF
  .withColumnRenamed("Index", "User_ID")  # Renommer une colonne
  .drop("_corrupt_record")                # Supprimer une colonne
)


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Écriture d'un flux</h2>

`DataFrame.writeStream` renvoie un `DataStreamWriter` utilisé pour configurer la sortie du flux.

Quelques paramètres pour la configuration de `DataStreamWriter` :
* Nom de la requête (facultatif) - nom unique parmi toutes les requêtes actives dans le SQLContext associé.
* Déclencheur (facultatif) - La valeur par défaut est `ProcessingTime(0`) et elle exécutera la requête aussi rapidement que possible.
* Répertoire de point de contrôle (facultatif pour les éviers pub/sub)
* Mode de sortie
* Évier de sortie (sink) : File Sink , Kafka Sink, Memory Sink, Foreach Sink, ...

Une fois la configuration terminée, nous pouvons déclencher le job avec un appel à `.start()`


<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Fenêtrage</h2>

Si nous utilisions un DataFrame statique pour produire un compte agrégé, nous pourrions utiliser `groupBy()` et `count()`.

Dans un streaming DataFrame, nous accumulons les comptes dans une fenêtre glissante, répondant à des questions comme "Combien d'enregistrements recevons-nous chaque seconde ?"

**Fenêtres glissantes** : Les fenêtres se chevauchent et un seul événement peut être agrégé dans plusieurs fenêtres.

**Fenêtres fixes** : Les fenêtres ne se chevauchent pas et un seul événement sera agrégé dans une seule fenêtre.

Illustration fenêtres glissante et fixe <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html" target="_blank">Guide de programmation de Structured Streaming</a>



### Déclencheurs

Le déclencheur spécifie quand le système doit traiter le prochain ensemble de données.

| Type de déclencheur                    | Exemple | Remarques |
|----------------------------------------|-----------|-------------|
| Non spécifié                           |  | _DEFAULT_ - La requête sera exécutée dès que le système aura terminé de traiter la requête précédente |
| Micro-lots à intervalle fixe           | `.trigger(Trigger.ProcessingTime("6 hours"))` | La requête sera exécutée en micro-lots et lancée aux intervalles spécifiés par l'utilisateur |
| Micro-lot unique                       | `.trigger(Trigger.Once())` | La requête exécutera _un seul_ micro-lot pour traiter toutes les données disponibles, puis s'arrêtera d'elle-même |
| Continu avec intervalle de point de contrôle fixe | `.trigger(Trigger.Continuous("1 second"))` | La requête sera exécutée en mode de traitement continu à faible latence, <a href="http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#continuous-processing" target = "_blank">mode de traitement continu</a>. 

Dans l'exemple ci-dessous, vous utiliserez un intervalle fixe de 3 secondes :

`.trigger(Trigger.ProcessingTime("3 seconds"))`

### Point de contrôle

Un <b>point de contrôle</b> stocke l'état actuel de votre job de streaming dans un système de stockage fiable tel que Azure Blob Storage ou HDFS. Il ne stocke pas l'état de votre job de streaming dans le système de fichiers local d'un nœud de votre cluster.

Avec les journaux d'écriture anticipée, un flux terminé peut être redémarré et il continuera là où il s'était arrêté.

Pour activer cette fonctionnalité, vous devez simplement spécifier l'emplacement d'un répertoire de point de contrôle :

`.option("checkpointLocation", checkpointPath)`


Points à considérer :
* Si vous n'avez pas de répertoire de point de contrôle, lorsque le job de streaming s'arrête, vous perdez tout l'état de votre job de streaming et lors du redémarrage, vous recommencez à zéro.
* Pour certains sinks, vous obtiendrez une erreur si vous ne spécifiez pas un répertoire de point de contrôle :<br/>
`analysisException: 'checkpointLocation must be specified either through option("checkpointLocation", ...)..`
* Notez également que chaque job de streaming doit avoir son propre répertoire de point de contrôle : pas de partage.


### Modes de Sortie

| Mode   | Exemple | Remarques |


| **Complet** | `.outputMode("complete")` | La table de résultats mise à jour entière est écrite dans le récepteur. L'implémentation individuelle du récepteur décide comment gérer l'écriture de la table entière. |


| **Ajout** | `.outputMode("append")`     | Seules les nouvelles lignes ajoutées à la table de résultats depuis le dernier déclenchement sont écrites dans le récepteur. |


| **Mise à jour** | `.outputMode("update")`     | Seules les lignes de la table de résultats qui ont été mises à jour depuis le dernier déclenchement seront sorties dans le récepteur. Depuis Spark 2.1.1 |

Dans l'exemple ci-dessous, nous écrivons dans un répertoire Parquet qui ne supporte que le mode `append` :

`dsw.outputMode("append")`



### Éviers (sink) de Sortie

`DataStreamWriter.format` accepte les valeurs suivantes, entre autres :

| Évier de Sortie | Exemple                                          | Remarques |
| --------------- | ------------------------------------------------ | --------- |
| **Fichier**     | `dsw.format("parquet")`, `dsw.format("csv")`...  | Déverse la Table de Résultats dans un fichier. Supporte Parquet, json, csv, etc. |
| **Kafka**       | `dsw.format("kafka")`      | Écrit la sortie dans un ou plusieurs sujets dans Kafka |
| **Console**     | `dsw.format("console")`    | Imprime les données dans la console (utile pour le débogage) |
| **Mémoire**     | `dsw.format("memory")`     | Met à jour une table en mémoire, qui peut être interrogée via Spark SQL ou l'API DataFrame |
| **foreach**     | `dsw.foreach(writer: ForeachWriter)` | C'est votre "échappatoire", vous permettant d'écrire votre propre type d'évier. |
| **Delta**       | `dsw.format("delta")`     | Un sink propriétaire |

Dans l'exemple ci-dessous, nous allons ajouter des fichiers à un répertoire Parquet et ensuite Delta et spécifier son emplacement avec cet appel :

`.format("parquet").start(outputPathDir)`

`.format("delta").start("db.table")`

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Exemple Streaming</h2>

Dans la cellule ci-dessous, nous écrivons des données d'une requête de streaming vers `outputPathDir`.

Note :

1. Nous donnons un nom à la requête via l'appel à `.queryName`

2. Spark commence à exécuter des jobs une fois que nous appelons `.start`

3. L'appel à `.start` renvoie un objet `StreamingQuery`

In [ ]:
outputPathDir = "/.../veille/output.parquet" # Sous-répertoire résultats du streaming
checkpointPath = "/.../output.parquet.checkpoint"    
# Définition de la requête de streaming
streamingQuery = (streamingDF                     
  .writeStream                                    
  .queryName("stream_1p")                         
  .trigger(processingTime="3 seconds")            # déclencheur pour exécuter un micro-lot toutes les 3 secondes
  .format("parquet")                              # destination au format Parquet
  .option("checkpointLocation", checkpointPath)   # fichiers de point de contrôle
  .outputMode("append")                           # seules les nouvelles données seront écrites dans le fichier
  .start(outputPathDir)                           # Démarre la requête et envoie les résultats dans le répertoire spécifié
)


25/12/15 13:20:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
spark.sql("create database if not exists outputDelta;")


DataFrame[]

In [ ]:
#outputPathDir = "/mnt/tmp/output.parquet" 
#checkpointPath = "/mnt/tmp/output.delta.checkpoint"    

outputPathDir = "/..../veille/output.parquet" # Sous-répertoire résultats du streaming
checkpointPath = "/...../output.parquet.checkpoint"


# Définition de la requête de streaming
streamingQuery = (streamingDF                     
  .writeStream                                    
  .queryName("stream_2p")                         
  .trigger(processingTime="3 seconds")            
  .format("delta")                             
  .option("checkpointLocation", checkpointPath)  
  .outputMode("append")                           
  .toTable("outputDelta.table")                           
)


25/12/15 13:25:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


25/12/15 13:25:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/12/15 13:25:50 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000} milliseconds, but spent 3117 milliseconds


In [ ]:
spark.sql("SELECT * FROM outputDelta.table").show(truncate=False)


%md

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Script complet</h2>

In [ ]:
# Définir le shéma de données
dataSchema = "Arrival_Time long, Creation_Time long, Device string, Index long, Model string, User string, gt string, x double, y double, z double"

# Lecture source de données
dataPath = "/databricks-datasets/definitive-guide/data/activity-data"
initialDF = (spark
  .readStream                       # Renvoie DataStreamReader
  .option("maxFilesPerTrigger", 1)  # Forcer le traitement d'un seul fichier par déclencheur
  .schema(dataSchema)               # Requis pour tous les DataFrames en streaming
  .json(dataPath)                   # Le répertoire source du flux et le type de fichier
)

# Petite transformation
streamingDF = (initialDF
  .withColumnRenamed("Index", "User_ID")  # Choisir un nom de colonne "meilleur"
  .drop("_corrupt_record")                # Supprimer une colonne inutile
)


# Ecriture du streaming dans un fichier au format Parquet
outputPathDir = "/mnt/tmp/output.parquet" 
checkpointPath = "/mnt/tmp/output.parquet.checkpoint"  
# Définition de la requête de streaming
streamingQuery = (streamingDF                     
  .writeStream                                    
  .queryName("stream_1p")                       
  .trigger(processingTime="3 seconds")            
  .format("parquet")                            
  .option("checkpointLocation", checkpointPath)   
  .outputMode("append")                           
  .start(outputPathDir)                           
)


# Ecriture du streaming dans une table Delta
outputPathDir = "/mnt/tmp/output.parquet" # Sous-répertoire où les résultats du streaming seront enregistrés au format Parquet
checkpointPath = "/mnt/tmp/output.delta.checkpoint"    # Sous-répertoire pour les fichiers de point de contrôle
# Définition de la requête de streaming
streamingQuery = (streamingDF                     
  .writeStream                                    
  .queryName("stream_2p")                         
  .trigger(processingTime="3 seconds")            
  .format("delta")                             
  .option("checkpointLocation", checkpointPath)  
  .outputMode("append")                           
  .toTable("outputDelta.table")                           
)


%md
Liste flux actifs

In [13]:
for stream in spark.streams.active:      # Loop over all active streams
    print(" {} ({})".format(stream.name, stream.id))


%md
Arretter les flux actifs

In [14]:
for stream in spark.streams.active:
  stream.stop()


%md

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Résumé</h2>

Nous utilisons `readStream` pour lire les entrées en streaming à partir de diverses sources et créer un DataFrame.

Rien ne se passe jusqu'à ce que nous invoquions `writeStream` ou `display`.

En utilisant `writeStream`, nous pouvons écrire vers une variété de récepteurs de sortie. En utilisant `display`, nous dessinons des graphiques en barres, des graphiques et d'autres types de tracés en direct dans le notebook.

%md

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Questions de révision</h2>

**Q:** Que font `readStream` et `writeStream` ?<br>


**Q:** Que produit `display` s'il est appliqué à un DataFrame créé via `readStream` ?<br>

**Q:** Lorsque vous exécutez une commande de flux d'écriture, que fait cette option `outputMode("append")` ?<br>

**Q:** Que se passe-t-il si vous ne spécifiez pas `option("checkpointLocation", pointer-to-checkpoint directory)` ?<br>


**Q:** Comment visualiser la liste des flux actifs ?<br>


**Q:** Comment vérifier si `streamingQuery` est en cours d'exécution (sortie booléenne) ?<br>

%md

%md

%md

%md

%md

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Questions de révision</h2>

**Q:** Que font `readStream` et `writeStream` ?<br>
**A:** `readStream` crée un DataFrame en streaming.<br>`writeStream` envoie les données en streaming vers un récepteur de sortie.

**Q:** Que produit `display` s'il est appliqué à un DataFrame créé via `readStream` ?<br>
**A:** `display` envoie les données en streaming vers un graphique EN DIRECT !

**Q:** Lorsque vous exécutez une commande de flux d'écriture, que fait cette option `outputMode("append")` ?<br>
**A:** Cette option prend les valeurs suivantes et leurs significations respectives :
* <b>append</b> : ajouter uniquement de nouveaux enregistrements au récepteur de sortie
* <b>complete</b> : réécrire la sortie complète - applicable aux opérations d'agrégation
* <b>update</b> : mettre à jour les enregistrements modifiés sur place

**Q:** Que se passe-t-il si vous ne spécifiez pas `option("checkpointLocation", pointer-to-checkpoint directory)` ?<br>
**A:** Lorsque le travail de streaming s'arrête, vous perdez tout l'état de votre job de streaming et lors du redémarrage, vous recommencez à zéro.

**Q:** Comment visualiser la liste des flux actifs ?<br>
**A:** Invoquez `spark.streams.active`.

**Q:** Comment vérifier si `streamingQuery` est en cours d'exécution (sortie booléenne) ?<br>
**A:** Invoquez `spark.streams.get(streamingQuery.id).isActive`.